# Google Cloud Platform - Using Machine Learning APIs  ).

This is an upgraded Python revision of [this notebook](https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/CPB100/lab4c/mlapis.ipynb).

This notebook originally was being processed using DataLab on the Google Cloud Platform.  This particular incarnation of the notebook is for running on Google Colaboratory which I am trying out for the first time.

### Security

First things first - we need to authenticate against the Google Cloud APIs.

#### Getting a Google API Credential.

First, visit <a href="http://console.cloud.google.com/apis">API console</a>, choose "Credentials" on the left-hand menu.  Choose "Create Credentials" and generate an API key for your application. You should probably restrict it by IP address to prevent abuse, but for now, just  leave that field blank and delete the API key after trying out this demo.

Then, when you have your key, you will enter it in this first executable cell:

In [0]:

'''
!pip install -q cloudstorage
!pip install -q webapp2
!pip install --upgrade numpy
!pip install six==1.10.0
!pip install -q datalab
!pip install --upgrade gcs-client
!pip install --upgrade google-cloud-storage
!pip install --upgrade google-api-python-client
!pip install opencv-python
!pip install gcsfs
'''

'\n!pip install -q cloudstorage\n!pip install -q webapp2\n!pip install --upgrade numpy\n!pip install six==1.10.0\n!pip install -q datalab\n!pip install --upgrade gcs-client\n!pip install --upgrade google-cloud-storage\n!pip install --upgrade google-api-python-client\n!pip install opencv-python\n!pip install gcsfs\n'

In [2]:
import getpass

APIKEY = getpass.getpass()

··········


In [3]:
import sys
import json
import argparse
import googleapiclient.discovery
import pandas as pd
import skimage.data
from skimage import img_as_float
import numpy as np
import pandas as pd
import os
import sys
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
%matplotlib inline
import io


import sys
import json
import argparse
import googleapiclient.discovery
import pandas as pd
import skimage.data
from skimage import img_as_float
import numpy as np
import pandas as pd
import os
import sys
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
%matplotlib inline
import io
from google.cloud import storage


import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
from keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
import matplotlib.image as mpimg


Using TensorFlow backend.


In [0]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

# Create the service client.
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')




In [0]:
BUCKET='wpiopenimageskaggle'
BUCKET_TUNE='mlclassifiertuning'

METHODS

In [0]:
def create_service():
    return googleapiclient.discovery.build('storage', 'v1')
def download_content(filename,bucket):
    from google.cloud import storage
    client = storage.Client('MLRobustClassifier')
    bucket = client.get_bucket(bucket)
    blob = storage.Blob(filename, bucket)
    content = blob.download_as_string()
    return(content)
def plot_image(filename):
  from google.cloud import storage
  client = storage.Client('MLRobustClassifier')
  bucket = client.get_bucket(BUCKET)
  blob = storage.Blob(filename, bucket)
  content = blob.download_as_string()
  img = Image.open(io.BytesIO(content))
  import matplotlib.pyplot as plt
  plt.imshow(img)
  plt.show()
def list_bucket_pf(bucket, pf):
    """Returns a list of metadata of the objects within the given bucket."""
    service = create_service()

    # Create a request to objects.list to retrieve a list of objects.
    fields_to_return = 'nextPageToken,items(name,size,contentType,metadata(my-key))'
    req = service.objects().list(bucket=bucket, fields=fields_to_return, prefix=pf)  # returns everything. UrbanSound is top dir in bucket

    all_objects = []
    # If you have too many items to list in one request, list_next() will
    # automatically handle paging with the pageToken.
    while req:
        resp = req.execute()
        all_objects.extend(resp.get('items', []))
        req = service.objects().list_next(req, resp)
    return all_objects
def process_image(filename, BUCKET):
  from google.cloud import storage
  client = storage.Client('MLRobustClassifier')
  bucket = client.get_bucket(BUCKET)
  blob = storage.Blob(filename, bucket)
  content = blob.download_as_string()
  img = Image.open(io.BytesIO(content)).convert('L')
  img=np.array(img)
  return(img)
def CreateDictLabels(Points):
    dictOfLabels = {}
    for point in Points:
        labelkey = point['annotations'][0]['label'] #get the label
        if labelkey not in dictOfLabels:
            dictOfLabels.setdefault(labelkey, [])
        dictOfLabels[labelkey].append(point['id'])
    return dictOfLabels
def CreateBatchOfImages(batchSize, labelDict, bucket,imageInputPath):
    #global imageInputPath
    ImageLabelDict = {}
    SuccessNum = 0
    for index in range(batchSize):
        for labelkey in labelDict.keys():
            if labelkey not in ImageLabelDict:
                ImageLabelDict.setdefault(labelkey, [])

            #force add one image, keep trying until one has been added, protects against corrupted images
            while len(labelDict[labelkey]) != 0 and  len(labelDict[labelkey]) <2000:
                try:
                    imageId = labelDict[labelkey].pop(0)
                    uriInp = imageInputPath + imageId + ".jpg"
                    if index%50==0:
                      print(index)
                    '''
                    with _open_file_read_binary(uriInp) as f:
                        image_bytes = f.read()
                        img = Image.open(io.BytesIO(image_bytes))
                    '''

                    #img= imageio.imread(uriInp)
                        #img = Image.open(io.BytesIO(image_bytes))
                    imgArr =  process_image(uriInp,bucket)
                except: #find image exception error for better style
                    continue
                else:
                    ImageLabelDict[labelkey].append([imgArr, imageId])
                    break
        #make sure all have nonzero number of images left
 
        SuccessNum += 1
    return ImageLabelDict, labelDict, SuccessNum

# IMPORT txt

In [7]:
from google.colab import files
uploaded = files.upload()
file='PointAnnotationsSet256x256.txt'
points = json.loads(uploaded[file].decode('utf-8')

Saving PointAnnotationsSet256x256.txt to PointAnnotationsSet256x256 (2).txt


In [8]:
from google.colab import files
uploaded = files.upload()
tuning_images='tuning_images256x256.csv'
tuning_labels='tuning_labels.csv'
tuning_points='PointAnnotationsSet_Tuning.txt'
points_tune = json.loads(uploaded[tuning_points].decode('utf-8'))

Saving PointAnnotationsSet_Tuning.txt to PointAnnotationsSet_Tuning (2).txt


In [22]:
#IMPORT DATA
classes=[]
IdsFromLabels = CreateDictLabels(points)
idsToRemoveFromEachBatch = IdsFromLabels.copy()
for val in IdsFromLabels.items():
  print(len(val[1]))
  classes.append(val[0])
  


17473
2148
1460
4554
6021
892
544
633


In [68]:
imageInputPathTrain='Imagefiles256x256/'
ImageLabelDict, idsToRemoveFromEachBatch, SuccessNum = CreateBatchOfImages(1000, idsToRemoveFromEachBatch, BUCKET,imageInputPathTrain)

0
50
100
150
200
250
300
350
400
450


In [66]:
#IMPORT DATA
IdsFromLabels_tune = CreateDictLabels(points_tune)
idsToRemoveFromEachBatch_tune = IdsFromLabels_tune.copy()
remove=[]
for val in IdsFromLabels_tune.items():
  if val[0] not in classes:
    remove.append(val[0])
for val in remove:  
  del IdsFromLabels_tune[val]
  
for val in IdsFromLabels_tune.items():
  print(val[0])
  print(val[1])


/m/01g317
['63424f32615631752b33383d', '2b6d4e7341712f704268593d', '4a3774367172792f6278773d', '2f414e7a6a44362b7364343d', '4d7973444169414d3148303d', '655a695268676875505a733d', '434b6b74585375546e506f3d', '4a3655416266617a30566b3d', '6c764f44574475705457673d', '71744c645270563876466b3d', '6f31432f6b5559705873553d', '6c4a4147754570397352633d', '3164376d766570626f32593d', '6e4b5062425971494a79303d', '36655548735946654a72383d', '346455452b49594d334f4d3d', '4a546b654f4641306461343d', '3166494744374a663636673d', '5a3357763955566b6a35383d', '65657832307441694865383d', '622f2b54686f37647739633d', '43327a595574514e62654d3d', '6544773735567645444d6b3d', '704b654756545a5a352f733d', '326346556f4c67694e62493d', '43397a79583147555a474d3d', '5075567644596b735657733d', '62477a514c4b5078744f633d', '55366854692b63746a55453d', '795166444d7a3373546a413d', '4c657844463956365366303d', '642b453738366e6a6d39383d', '4969383250465362785a593d', '374f5172776d686b6d4a773d', '64544e4a346857482f47673d', '4d4d474a

In [62]:
 test='Imagefiles256x256/63424f32615631752b33383d.jpg'
 result=process_image(test, BUCKET_TUNE)
print(result)

[[121 121 121 ... 192 192 192]
 [121 121 121 ... 191 192 192]
 [121 121 121 ... 191 191 192]
 ...
 [149 127 116 ... 184 110  72]
 [147 140 124 ... 178 130  79]
 [126 151 147 ... 158 134  84]]


In [69]:
imageInputPathTrain='Imagefiles256x256/'
idsToRemoveFromEachBatch_tune = IdsFromLabels_tune.copy()

ImageLabelDict_tune, idsToRemoveFromEachBatch_tune, SuccessNum = CreateBatchOfImages(100, idsToRemoveFromEachBatch_tune, BUCKET_TUNE,imageInputPathTrain)

0
0
0
0
0
50
50


In [70]:
print(idsToRemoveFromEachBatch_tune)
for val in keys:
  imgs=ImageLabelDict.get(val)
  labels.append(val)
  for img in imgs:

{'/m/01g317': ['464c7a7874395563534d6b3d', '6c79506b714e53496b50673d', '6a39326b6a4a6f546d62593d', '594a506c6a6a59337356383d', '52615568596478675254413d', '50516e75345056434a36733d', '424a5677325147585844773d', '333863456a4b796b6f61493d', '31552f30666c4f6d4341733d', '6959742b51396d482b4b513d', '3153786b715676455431553d', '6a5a452b2f5473706651493d', '736f303344522b493158493d', '526e317865554369704b633d', '57504f69534764416d6b733d', '414c45587a697868704d493d', '6644626a4f5863355932553d', '2b52586f4a6a52707764493d', '4a694b3247392b436c36633d', '6632584a6848336b577a513d', '72734436664562457253593d', '527647475836743439646b3d', '2b38364476612f6c4558383d', '6542655148774d647677593d', '727a546f62432b443042513d', '592f4c76674667416245413d', '374d396242616f354e4e593d', '4e4467384f6634646d32493d', '4267472f4f516a597158303d', '7642336c354b434a645a453d', '72455170564633536453343d', '694d58716f6b542b6650413d', '4a57574746396f617773493d', '4f7a547a49743861696e673d', '6f684b59366448697765413d'], '/m/

In [0]:

'''
##USE THIS WHEN SPLITTING THE TRAINING FOR TEST/TRAIN
keys=list(ImageLabelDict.keys())
data=[]
labels=[]
label=0
for val in keys:
  imgs=ImageLabelDict.get(val)
  for img in imgs:
        vec=img[0]
        data.append(vec)
        labels.append(label)
  label+=1
        
    ########
    ##
    ##Split data intro training and testing wtih 0.75 train vs 0.25 test
    ########
        
total=len(data)
train_size=round(total*0.75)
test_size=total-train_size
import random
random.seed(4)



train_ind=np.random.choice(len(labels),train_size,replace=False)
test_ind=np.setdiff1d(list(range(0, total)),train_ind)


labels_test=np.array(labels)
data_test=np.array(data)
    
X_train=data_test[np.where(train_ind)]
X_test=data_test[np.where(test_ind)]
Y_train=labels_test[np.where(train_ind)]
Y_test=labels_test[np.where(test_ind)]

img_len=256
img_width=256
# We reshape the input data to have a depth of 1 (grey scale)
if keras.backend.image_data_format() == 'channels_first':
        X_train = X_train.reshape(X_train.shape[0], 1, img_len, img_width)
        X_test = X_test.reshape(X_test.shape[0], 1, img_len, img_width)
        input_shape = (1, img_len, img_width)
else:
        X_train = X_train.reshape(X_train.shape[0], img_len, img_width, 1)
        X_test = X_test.reshape(X_test.shape[0], img_len, img_width, 1)
        input_shape = (img_len, img_width, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Then we normalize it so that the values are between 0 and 1
X_train /= 255
X_test /= 255
'''

In [0]:
##USE THIS WHEN SPLITTING THE TRAINING FOR TEST/TRAIN
X_train=[]
X_test=[]
Y_train=[]
Y_test=[]



###CONVERT FORMAT OF TRAINING DATA
keys=list(ImageLabelDict.keys())
data=[]
labels=[]
label=0
for val in keys:
  imgs=ImageLabelDict.get(val)
  labels.append(val)
  for img in imgs:
        vec=img[0]
        X_train.append(vec)
        Y_train.append(label)
  label+=1

In [80]:
  imgs=ImageLabelDict_tune.get('/m/019jd')
  print(imgs)

None


In [78]:
print(keys)

['/m/01g317', '/m/0cgh4', '/m/019jd', '/m/0bt9lr', '/m/07j7r', '/m/04bcr3', '/m/0jyfg', '/m/03vt0']


In [85]:
###CONVERT FORMAT OF TESTING DATA    
keys_raw=list(ImageLabelDict_tune.keys())
keys_clean=[]
for val in keys:
  imgs=ImageLabelDict_tune.get(val)
  if len(imgs)>0:
    keys_clean.append(val)



TypeError: ignored

In [49]:
for val in keys:
  imgs=ImageLabelDict_tune.get(val)
  for img in imgs:
        if imgs!= None:
          vec=img[0]
          X_test.append(vec)
          label=labels.index(val)
          print(label)
          Y_test.append(label)

[]


In [0]:
   # Defining the model
    model = Sequential()
    model.add(Conv2D(32, 3, 3, activation='relu', input_shape=(img_len, img_width, 1)))
    # 32 is the number of convolutional filters to use. Frist 3 is the number of rows in each convolution kernel and second 3 is the number of columns in each kernel.


    model.add(Conv2D(32, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    # MaxPooling2D is a way to reduce the number of parameters in our model by sliding a 2x2 pooling filter across the previous layer and taking the max of the 4 values in the 2x2 filter.

    model.add(Dropout(0.25))
    # Dropout regularizes the model and prevents overfitting

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    # Final layer has the output size of 10 to correspond to the number of classes

    model.summary()
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit model to training data
    model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1)

    # Evaluate the model on test data
    score = model.evaluate(X_test, Y_test, verbose=0)
    print("test loss:", score[0])
    print("test accuracy:",  score[1])
    # score[0] gives you the test loss and score[1] gives you the accuracy

    tbCallBack = TensorBoard(log_dir='./log', histogram_freq=1, write_graph=True, write_grads=True, batch_size=32, write_images=True)

    # We can use a call back to look into the internal state of the model during training
    model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, Y_test), callbacks=[tbCallBack])





/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(256, 256,...)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 32)      320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 252, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 126, 126, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 508032)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65028224  
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [0]:
print(len(Y_test))
print(np.unique(Y_test))
print(np.unique(Y_train))
x_

767
[2]
[2 5 6]


In [0]:
Y_train=pd.DataFrame(Y_train)
pd.value_counts(Y_train[0])

2    1000
5     892
6     409
Name: 0, dtype: int64

***BELOW THIS POINT IS CODE TRIED THAT DOESN'T WORK KEPT FOR REFERENCE***